In [1]:
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url

from selenium import webdriver

#options = webdriver.ChromeOptions()
#options.add_argument('headless')
#options.add_argument('window-size=1920x1080')
#options.add_argument("disable-gpu")
# 혹은 options.add_argument("--disable-gpu")


driver = webdriver.Chrome("E:/chromedriver_win32/chromedriver.exe")

import time

#인스타 접속, 
driver.get('https://www.instargram.com')
driver.find_element_by_xpath('//*[@id="details-button"]').click()
driver.find_element_by_xpath('//*[@id="proceed-link"]').click()
time.sleep(5) #시간을 넉넉히 설정합니다.

#로그인에 필요한 사항을 브라우저 제어를통해 입력합니다.
email = 'flora0825@naver.com'   ### 사용할 계정 정보 입력
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'dldudwn8' ### 비번 정보 수정
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()


time.sleep(3)
word = "플레이모빌" #검색할 해시태그. 띄어쓰기를 사용하면 안됩니다
url = insta_searching(word)
driver.get(url)
time.sleep(2)

#첫게시물을 여는 함수를 정의합니다
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)


import re
from bs4 import BeautifulSoup

#내용을 가져오는 함수를 정의합니다.
def get_content(driver):
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    

    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
    # 정규표현식을 활용하여 해시태그를 가져옵니다.
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # 작성일자를 가져옵니다.
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # 좋아요 수를 가져옵니다.
    try:
        like = soup.select('a.zV_Nj')[0].text[4:-1]   
    except:
        like = 0
    # 위치정보를 가져옵니다.
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
    # 수집한 정보를 리스트로 저장합니다.
    data = [content, date, like, place, tags]
    return data




#다음게시글을 여는 함수를 정의합니다.
def move_next(driver):

    right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)


#함수를 차례대로 실행합니다.
select_first(driver)
results = [ ]
target = 5    # 크롤링할 게시글 수
for i in range(target):
    # 게시글 수집에 오류 발생시 2초 대기후 다음글로 넘어갑니다.
    try:
        data = get_content(driver)    # 게시글 정보 가져오기
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)
    

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div._9AhH0"}
  (Session info: chrome=88.0.4324.182)


In [ ]:
#result적당히 출력해보기
print(results[:2])


#데이터프레임 만들고 엑셀로 저장하기
import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags']
results_df.to_excel('crawling_sample.xlsx', index=False) 